In [2]:
#importando as bibliotecas necessárias 
import pandas as pd

In [3]:
'''
- Leitura dos dados dos arquivos CSV e JSON - utilizando o parametro lines = True pois temos 1 JSON por linha e usando o 
r na frente da string para ignorar os caracteres de escape.

- Aqui aproveito também dividir a coluna de event_data em colunas individuais para que seja possivel fazer o join das 
tabelas de forma mais fácil e precisa e poder realizar as análises a partir de uma única tabela final. Faço isso tanto 
para o df mostrado quanto para o clicado e em seguida dropo a coluna event_data, pois ela não será mais necessária e uso o
parametro axis=1 para indicar que é uma coluna e não uma linha (axis =0)

'''

mostrado = pd.read_json((r'..\case_meli\prints.json'), lines=True)
mostrado = mostrado.assign(
    position=mostrado['event_data'].apply(lambda x: x['position']),
    value_prop=mostrado['event_data'].apply(lambda x: x['value_prop'])
).drop('event_data', axis=1)


In [22]:
'''
- Criando a coluna 'clicked' no df clicado para indicar se um value prop foi clicado ou não. Aqui atribuo o valor 1 
(clicado) para todas as linhas sem distinção, pois entende-se que nesta origem de dados só temos informações de value props 
que foram clicados, então todos recebem o valor 1.

'''

clicado = pd.read_json(r'..\case_meli\taps.json', lines=True)
clicado = (clicado
        .assign(position=clicado['event_data'].apply(lambda x: x['position']),
                value_prop=clicado['event_data'].apply(lambda x: x['value_prop']))
        .drop('event_data', axis=1)
        .assign(clicked=1))


In [28]:
'''
- trocando o nome da coluna pay_date para day para facilitar no join das tabelas
'''
pagamento = pd.read_csv(r'..\case_meli\pays.csv')
pagamento = pagamento.rename(columns={'pay_date': 'day'}) 

In [66]:
'''
Fazendo o join das 3 tabelas. Usandoo outer join para manter todos os registros de todas as tabelas, mesmo que um registro
não esteja em uma das três, ele será mantido no df final. As colunas utilizadas como chave são as colunas que deveriam ser
únicas para cada registro para identificar um registro único.
'''
df_joined = (
    mostrado
    .merge(pagamento, how='outer', on=['user_id', 'value_prop', 'day'])
    .merge(clicado, how='outer', on=['user_id', 'value_prop', 'day', 'position'])
)

In [33]:
'''
Para a etapa de análise, crio variáveis com as datas que precisamos para as análises:
- data inicial da última semana (de acordo com a última data disponível na tabela)
- data inicial da terceira semana a partir do inicio a última semana
'''
ult_sem = pd.to_datetime(df_joined['day']).max() - pd.Timedelta(days=7)
tres_sem_atras = ult_sem - pd.Timedelta(weeks=3)


In [72]:
'''
Filtrando os prints da última semana e fazendo o preenchimento de nulos na coluna clicked com o valor 0 (não clicado), 
pois após realizar o join das tabelas, os registros onde não houveram match com a tabela clicado, entende-se que não
houveram cliques.

Crio uma cópia usando o .copy() para desvincular do df original
'''
prints_ult_sem = df_joined[pd.to_datetime(df_joined['day']) >= ult_sem].copy()
prints_ult_sem['clicked'] = prints_ult_sem['clicked'].fillna(0).astype('int')

In [74]:
'''
print da analise
'''
print(prints_ult_sem)

                day  user_id  position        value_prop   total  clicked
381063   2020-11-23    69000       0.0  credits_consumer   98.59        0
381064   2020-11-23    69000       1.0        link_cobro  152.51        0
381065   2020-11-23    69000       2.0         transport     NaN        0
381066   2020-11-23    69000       3.0           prepaid     NaN        0
381067   2020-11-23    66521       0.0           prepaid     NaN        0
...             ...      ...       ...               ...     ...      ...
1246720  2020-11-30    74524       NaN        send_money   17.76        0
1246721  2020-11-30    42594       NaN         transport  111.02        0
1246722  2020-11-30     1338       NaN         transport    0.81        0
1246723  2020-11-30     1338       NaN  credits_consumer   34.71        0
1246724  2020-11-30    99962       NaN           prepaid   11.76        0

[334550 rows x 6 columns]


In [75]:
''' 
Filtrando os prints das últimas 3 semanas antes da última semana
'''
tres_sem_atras = df_joined[(pd.to_datetime(df_joined['day']) >= tres_sem_atras) & (pd.to_datetime(df_joined['day']) < ult_sem)]

In [87]:
'''
Agrupando os dadospara mostrar a soma da quantidade de cliques em cada value prop

Colocando o tipo da variável para inteiro, pois sempre após um cálculo o python retorna uma variável como float e nesse 
caso queremos um inteiro.
'''
visualizado = tres_sem_atras.groupby('value_prop')['clicked'].sum().astype(int)

In [88]:
'''
print da analise
'''
visualizado

value_prop
cellphone_recharge    5214
credits_consumer      5230
link_cobro            5278
point                 5193
prepaid               5141
send_money            5098
transport             5217
Name: clicked, dtype: int32

In [89]:
'''
Agrupando os dados por usuário e por value prop para trazer as informações de quantos cliques cada usuário fez em cada 
dos value props

Trocando o tipo para inteiro também
'''
clicado = tres_sem_atras.groupby(['user_id', 'value_prop'])['clicked'].sum().astype(int)


In [90]:
'''
print da analise
'''
clicado

user_id  value_prop      
1        credits_consumer    0
         link_cobro          1
         point               0
         prepaid             0
         send_money          0
                            ..
100000   link_cobro          0
         point               0
         prepaid             0
         send_money          0
         transport           0
Name: clicked, Length: 507190, dtype: int32

In [100]:
'''
Agrupando os dados por usuário e value prop para trazer a informação de quantos pagamentos foi realizado pelo usuário em
cada value prop
'''
pago = tres_sem_atras.groupby(['user_id','value_prop']).agg({'total': 'count'})

In [117]:
'''
Trazendo os resultados de quantidade de pagamento
'''
pago

total
user_id value_prop             
1       credits_consumer      1
        link_cobro            0
        point                 0
        prepaid               0
        send_money            0
...                         ...
100000  link_cobro            0
        point                 0
        prepaid               1
        send_money            1
        transport             1

[507190 rows x 1 columns]

In [132]:
'''
Agrupando os dados por usuário e value prop para trazer a informação de valor total de pagamentos realizado pelo usuário em
cada value prop
'''
valor_pago = pago = tres_sem_atras.groupby(['user_id','value_prop']).agg({'total': 'sum'})

In [133]:
'''
print dos resultados
'''
valor_pago

total
user_id value_prop             
1       credits_consumer  37.92
        link_cobro         0.00
        point              0.00
        prepaid            0.00
        send_money         0.00
...                         ...
100000  link_cobro         0.00
        point              0.00
        prepaid           14.37
        send_money        19.31
        transport          2.33

[507190 rows x 1 columns]